## Imports, data load, metric function definition

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

In [2]:
X_train = np.load('X_train_surge.npz')
Y_train = pd.read_csv('Y_train_surge.csv')
X_test = np.load('X_test_surge.npz')

In [11]:
X_train['slp'][0].shape

(40, 41, 41)

In [10]:
import torch

def preprocessing(X, device):
    slps = torch.from_numpy(X['slp']).to(device)
    
    
    
    return x['id_seq'], slps, t_slps, 

In [9]:
for f in X_train.files:
    print(f)

id_sequence
slp
t_slp
t_surge1_input
surge1_input
t_surge2_input
surge2_input
t_surge1_output
t_surge2_output


In [8]:
X_train['t_surge1_output'][0]

array([-6.306624e+08, -6.306192e+08, -6.305760e+08, -6.305328e+08,
       -6.304860e+08, -6.304428e+08, -6.303996e+08, -6.303528e+08,
       -6.303096e+08, -6.302664e+08], dtype=float32)

In [7]:
X_test['t_slp'][0]

array([-6.297696e+08, -6.297588e+08, -6.297480e+08, -6.297372e+08,
       -6.297264e+08, -6.297156e+08, -6.297048e+08, -6.296940e+08,
       -6.296832e+08, -6.296724e+08, -6.296616e+08, -6.296508e+08,
       -6.296400e+08, -6.296292e+08, -6.296184e+08, -6.296076e+08,
       -6.295968e+08, -6.295860e+08, -6.295752e+08, -6.295644e+08,
       -6.295536e+08, -6.295428e+08, -6.295320e+08, -6.295212e+08,
       -6.295104e+08, -6.294996e+08, -6.294888e+08, -6.294780e+08,
       -6.294672e+08, -6.294564e+08, -6.294456e+08, -6.294348e+08,
       -6.294240e+08, -6.294132e+08, -6.294024e+08, -6.293916e+08,
       -6.293808e+08, -6.293700e+08, -6.293592e+08, -6.293484e+08],
      dtype=float32)

In [6]:
surge_train = np.array(Y_train)[:,1:]

In [22]:
X_train['t_slp'][1] - X_train['t_slp'][1][0]

array([     0.,  10816.,  21632.,  32384.,  43200.,  54016.,  64768.,
        75584.,  86400.,  97216., 108032., 118784., 129600., 140416.,
       151168., 161984., 172800., 183616., 194432., 205184., 216000.,
       226816., 237568., 248384., 259200., 270016., 280832., 291584.,
       302400., 313216., 323968., 334784., 345600., 356416., 367232.,
       377984., 388800., 399616., 410368., 421184.], dtype=float32)

In [20]:
X_train['t_surge1_input'][0]

array([-6.311088e+08, -6.310656e+08, -6.310188e+08, -6.309756e+08,
       -6.309288e+08, -6.308856e+08, -6.308424e+08, -6.307992e+08,
       -6.307524e+08, -6.307092e+08], dtype=float32)

In [23]:
X_train['surge1_input'][0]

array([-0.54593754,  0.13110517,  0.31880018,  0.19478741, -0.22752637,
       -0.4018146 , -0.87440383, -0.63978505, -1.0755056 , -1.712328  ],
      dtype=float32)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models

resnet_versions = {
    18 : models.resnet18,
    34 : models.resnet34,
    50 : models.resnet50,
}

class Conv2d(nn.Conv2d):
    """Weight standardisation
    (see https://arxiv.org/pdf/1903.10520.pdf and https://github.com/joe-siyuan-qiao/WeightStandardization)
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super(Conv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                 padding, dilation, groups, bias)

    def forward(self, x):
        weight = self.weight
        weight_mean = weight.mean(dim=1, keepdim=True).mean(dim=2,
                                  keepdim=True).mean(dim=3, keepdim=True)
        weight = weight - weight_mean
        std = weight.view(weight.size(0), -1).std(dim=1).view(-1, 1, 1, 1) + 1e-5
        weight = weight / std.expand_as(weight)
        return F.conv2d(x, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
    
class Network(nn.Module):
    
    def __init__(self, resnet_layers):
        self.resnet = resnet_versions[resnet_layers]()
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, _out_features_)
        # LSTM:
        # Add prediction time?
        # Learn to associate pressure_mapt to their timestamps
        
    def forward(self, ):

In [4]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

## Benchmark
Train using kNN of pressure fields at two instants in time, with 40 neighbours

In [5]:
nfields = 2; time_step_slp = 8
slp_train = []
slp_all = X_train['slp']
for i in range(5559):
    slp_train.append(np.ndarray.flatten(slp_all[i,-1]))
    for j in range(1,nfields):
        slp_train[-1] = np.concatenate( ( slp_train[-1], np.ndarray.flatten(slp_all[i,-1-j*time_step_slp]) ) )
slp_train = np.array(slp_train)

In [6]:
slp_test = []
slp_all_test = X_test['slp']
for i in range(509):
    slp_test.append(np.ndarray.flatten(slp_all_test[i,-1]))
    for j in range(1,nfields):
        slp_test[-1] = np.concatenate( ( slp_test[-1], np.ndarray.flatten(slp_all_test[i,-1-j*time_step_slp]) ) )
slp_test = np.array(slp_test)

In [7]:
tree = BallTree(slp_train)

In [8]:
surge_test_benchmark = []; k = 40
for i in range(509):
    dist, ind = tree.query([slp_test[i]], k=k)
    surge_test_benchmark.append(np.mean(surge_train[ind[0]], axis=0))
surge_test_benchmark = np.array(surge_test_benchmark)

In [9]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=surge_test_benchmark, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('Y_test_benchmark.csv', index_label='id_sequence', sep=',')